In [1]:
import argparse
import sys
import torch
import torch.nn.functional as F
import rdkit
import pandas as pd
sys.path.append("..")
from tqdm.auto import tqdm
from modeling_smi.models_storage import ModelsStorage
from modeling_smi.script_utils import add_sample_args, set_seed, add_train_args, read_smiles_csv
from modeling_smi.dataset import get_dataset

In [2]:
config_load='../checkpoints/zinc_mask/transformer_config.pt'
vocab_load='../checkpoints/zinc_mask/transformer_vocab.pt'
model_load='../checkpoints/zinc_mask/transformer_model.pt'

In [3]:
MODELS = ModelsStorage()
model = 'transformer'
model_config = torch.load(config_load)
model_vocab = torch.load(vocab_load)
model_state = torch.load(model_load)
model = MODELS.get_model_class(model)(model_vocab, model_config)
model.load_state_dict(model_state)
model = model.cuda()
model.eval()

Transformer(
  (encoder): Embedding(29, 128, padding_idx=25)
  (drop): Dropout(p=0.2, inplace=False)
  (encoder_blocks): Sequential(
    (0): Block(
      (ln1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (self_attn): CausalSelfAttention(
        (key): Linear(in_features=128, out_features=128, bias=True)
        (query): Linear(in_features=128, out_features=128, bias=True)
        (value): Linear(in_features=128, out_features=128, bias=True)
        (attn_drop): Dropout(p=0.2, inplace=False)
        (resid_drop): Dropout(p=0.2, inplace=False)
        (proj): Linear(in_features=128, out_features=128, bias=True)
      )
      (mlp): Sequential(
        (0): Linear(in_features=128, out_features=512, bias=True)
        (1): GELU()
        (2): Linear(in_features=512, out_features=128, bias=True)
        (3): Dropout(p=0.2, inplace=False)
      )
    )
    (1): Block(
      (ln1): LayerNorm((128,), eps=1e-0

In [52]:
test_load='../data/zinc_test.csv'
input_str='COc1c<mask>cc(Cn2nc3oc(=O)cc(C)c3c2C)c1'
tensors = model.string2tensor(input_str)
tensors = tensors.unsqueeze(0)
tensors.shape

torch.Size([1, 36])

In [53]:
output=model(tensors,0).squeeze()
print(output.shape)
output = F.softmax(output,dim=-1)
output=torch.multinomial(output, 1)
output=output.squeeze()
print(output.shape)
output_str=model.tensor2string(output)

torch.Size([36, 29])
torch.Size([36])


In [54]:
output_str

'COc1cccc(Cn2nc3oc(=O)cc(C)c3c2C)c1c'